In [ ]:
import pandas as pd
from pymongo import MongoClient
from geopy.distance import VincentyDistance
import datetime
import psycopg2
import json
import sys


client = MongoClient('mongodb://ec2-34-248-97-137.eu-west-1.compute.amazonaws.com:27017')
db = client['C8Geo']
collection = db['locations']


conn_string = "host='c-sec-prod.cilede3fhork.eu-west-1.rds.amazonaws.com' dbname='c_db' user='mesut' password='TeW15WQBfA7i'"
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()




def GetLocAudSegments(Latitude, Longitude, Radius, AudienceGroup):
    
    TargetLocation = Latitude, Longitude
    
    # IDs of Users Within Radius meters of Target Location
    UsersWithin = []

    for doc in (collection.find({'created_at' : {'$exists' : True}, 'created_at' : {'$gte' : datetime.datetime(2017, 4, 6)}, 
                         'created_at' : {'$lte' : datetime.datetime.now()}}, projection={'_id' : False, 'userId' : True, 
                                                             '_source.coords.latitude' : True, '_source.coords.longitude' : True})):


        CurrentLoc = doc['_source']['coords']['latitude'], doc['_source']['coords']['longitude']

        if (VincentyDistance(TargetLocation, CurrentLoc).meters <= Radius) & (not doc['userId'] in UsersWithin):

            UsersWithin.append(doc['userId'])
            
            
    
    
    # Users' Top Attributions
    cursor.execute('select id, name from audiences')
    AudIdName = pd.DataFrame(cursor.fetchall())
        
    
    ID2Name = {}
    for _, (AudienceID, AudienceName) in AudIdName.iterrows():
        ID2Name[AudienceID] = AudienceName   
    
    
    AudienceCounts = {}
    AudienceThresholds = json.load(open('/opt/ds/AudienceThresholds.json'))
    
    
    for User in UsersWithin:
            
        cursor.execute('select mean, audience_id from user_audiences_mean where user_id = %s and audience_id in (select id from audiences where audience_group_id = %s)', (User, AudienceGroup))
        UserValues = pd.DataFrame(cursor.fetchall())



        for _, (mean, audience_id) in UserValues.iterrows():
            try:
                if mean >= AudienceThresholds[ID2Name[int(audience_id)]]:

                    if int(audience_id) in AudienceCounts.keys():
                        AudienceCounts[int(audience_id)] += 1
                    else:
                        AudienceCounts[int(audience_id)] = 1
            except:
                continue



    AudienceCounts_DF = pd.DataFrame(columns=['Audience Name', '%'])
    for AudienceID, Count in AudienceCounts.items():
        AudienceCounts_DF = AudienceCounts_DF.append({'Audience Name' : ID2Name[AudienceID], '%' : 
                                                                              Count/len(UsersWithin)*100}, ignore_index=True)
    
    
    AudienceCounts_DF.sort_values(by='%', ascending=False).to_csv('/opt/ds/LocAudSegments_ForAG_' + AudienceGroup + '.csv')
    
    
    return None




if __name__ == '__main__':
    
    Latitude = float(sys.argv[1])
    Longitude = float(sys.argv[2])
    Radius = float(sys.argv[3])
    AudienceGroup = str(sys.argv[4])
    
    GetLocAudSegments(Latitude, Longitude, Radius, AudienceGroup)